In [1]:
# INSTALAR E IMPORTAR LIBRERIAS
#!pip install mysqlclient
#!pip install PyMySQL
#!pip install mysql-connector-python
import MySQLdb as msdb
import pymysql
import pandas as pd
import numpy as np
import mysql.connector as mysql
from mysql.connector import connection


In [2]:
# Funcion para conectar a la DB
def connect_to_db (host_name,username,puerto,password,db_name):
    try:
        conn= mysql.connect(host=host_name, user=username,port=puerto, password=password,database=db_name)
    except mysql.OperationalError as e:
        raise e
    else:
        print('Connected!')
    return conn

In [3]:
#Consejos al conectar con AWS
#Es importante para que conecte establecer los parametros de seguridad en AWS que permitan acceso al ip
#El puede conectar a solo la ip local o todas.


host_name="xxx.x.x.x" # Sustituir por ip de AWS o LOCALHOST
db_name= "prueba_db" # Cualquier nombre
username="root" #Cualquier usuario
password= "1234567890" #Cualquier clave
puerto= int(3306) #Puerto de la DDBB
conn= None #Inicialmente None

In [4]:
#Conectar a la DDBB
conn = connect_to_db (host_name,username,puerto, password,db_name)

Connected!


In [5]:
#Leer el archivo csv creado 
df = pd.read_csv('prueba_youtube_api_nanutria4.csv')#, index_col=0)
df.head()


,video_id,video_title,upload_date,view_count,like_count,favorite_count,comment_count
0,8Fb2GmLes04,El SIPDN Autopromocionarse para lograrlo con...,2022-03-15,15250,588,0,106
1,WtLhkxArl4E,El SIPDN Una guitarrista venezolana en Holly...,2022-03-08,16443,680,0,111
2,S-CGw_fWEfA,El SIPDN Cómo es la vida de un gamer profesi...,2022-03-01,17973,581,0,86
3,847qMLoON_c,El SIPDN Un podcast de campeonato mundial co...,2022-02-22,27552,1105,0,176
4,wDAfLRJDxbc,El SIPDN Sobreviviendo a la naturaleza y a l...,2022-02-15,42189,1600,0,512


In [6]:
#Crear Tabla
def create_table(curr):
    create_table_command=("""CREATE TABLE IF NOT EXISTS videos2(
                                video_id VARCHAR(255) PRIMARY KEY,
                                video_title TEXT NOT NULL,
                                upload_date DATE NOT NULL DEFAULT (CURRENT_DATE),
                                view_count INTEGER NOT NULL,
                                like_count INTEGER NOT NULL,
                                favorite_count INTEGER NOT NULL,
                                comment_count INTEGER NOT NULL)
                                """)
    
    curr.execute(create_table_command)  

In [7]:
# Revisa si el video existe
def check_if_video_exists(curr,video_id):
    query= (""" SELECT video_id FROM videos2 WHERE video_id = %s""")
    curr.execute(query,(video_id,))
    return curr.fetchone() is not None

In [8]:
#Actualiza las filas si el video existe
def update_row(curr, video_id,video_title, upload_date, 
               view_count, like_count, favorite_count, comment_count):
    query= (""" UPDATE videos2
            SET video_title = %s,
            upload_date = %s,
            view_count = %s,
            like_count = %s,
            favorite_count = %s,
            comment_count = %s
            WHERE video_id = %s;""")
    vars_to_update = (video_title, upload_date, view_count, like_count, 
                      favorite_count,comment_count, video_id)
    curr.execute(query,vars_to_update)


In [9]:
# Actualiza la DDBB
def update_db(curr,df):
    tmp_df = pd.DataFrame(columns=['video_id','video_title', 'upload_date', 'view_count','like_count',
                                  'favorite_count', 'comment_count'])
    for row in df.itertuples():
        if check_if_video_exists(curr, row[1]):# Si el video ya existe se actualiza
            update_row(curr, row[1],row[2],row[3],row[4],row[5],row[6],row[7])
        else: # Si el video no existe se añade a la tabla
            a={'video_id':row[1],'video_title':row[2], 'upload_date':row[3], 
               'view_count':row[4],'like_count':row[5],'favorite_count':row[6],
               'comment_count':row[7]}
            tmp_df = tmp_df.append(a, ignore_index=True)
    return tmp_df


In [10]:
#Comando para inserta a la tabla
def insert_into_table(curr,video_id,video_title, upload_date,view_count, 
                      like_count, favorite_count, comment_count):
    insert_into_videos = ("""INSERT INTO videos2 (video_id, video_title, upload_date, view_count,
                    like_count, favorite_count,comment_count) VALUES (%s,%s,%s,%s,%s,%s,%s);""")
    row_to_insert=(video_id,video_title, upload_date, 
                   view_count, like_count, favorite_count, comment_count)
    curr.execute(insert_into_videos,row_to_insert)

In [11]:
def append_from_df_to_db(curr,df):
    for row in df.itertuples():
        insert_into_table(curr,row[1],row[2],row[3],row[4],row[5],row[6],row[7])
        
    

In [12]:
# Para que este comando .cursor() funcione ya debe haberse creado una conexión a la DB
curr= conn.cursor()

In [13]:
create_table(curr)

In [14]:
new_video_df=update_db(curr,df)

In [15]:
new_video_df

,video_id,video_title,upload_date,view_count,like_count,favorite_count,comment_count
0,8Fb2GmLes04,El SIPDN Autopromocionarse para lograrlo con...,2022-03-15,15250,588,0,106
1,WtLhkxArl4E,El SIPDN Una guitarrista venezolana en Holly...,2022-03-08,16443,680,0,111


In [16]:
append_from_df_to_db(curr,new_video_df)

In [17]:
conn.commit()

In [ ]:
# Queda pendiente medir el tiempo que gasta itertuples e intentar convertirlo a np.where ¿?
